<a href="https://colab.research.google.com/github/ind-kum/M.L.-EVELUTION-PROJECTS/blob/master/Project_40_Data_Scientist_Sallary_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g6JuJLqByHwPHGz_VYRpoZdT8ItSq0iRotTkJG_ayqcVjmePhIzUN4
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv("/content/drive/MyDrive/DATASETS/Data_Scientist_Sallary_Dataset/Train_Dataset.csv")
test=pd.read_csv("/content/drive/MyDrive/DATASETS/Data_Scientist_Sallary_Dataset/Test_Dataset.csv")
train.head()

,Unnamed: 0,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,NaN,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [ ]:
#lets chcek the null values
train.isnull().sum()

Unnamed: 0                  0
experience                  0
job_description          4418
job_desig                   0
job_type                15005
key_skills                  1
location                    0
salary                      0
company_name_encoded        0
dtype: int64

In [ ]:
#Lets chcek the value counts of object type columns
for col in train.columns:
    if train[col].dtype=="object":
        print(train[col].value_counts())
        print()

5-10 yrs     1274
2-5 yrs      1188
3-8 yrs       922
2-7 yrs       832
4-9 yrs       678
             ... 
14-19 yrs       1
12-13 yrs       1
17-25 yrs       1
17-24 yrs       1
11-12 yrs       1
Name: experience, Length: 129, dtype: int64

Accenture Technology powers our clients businesses with innovative technologies established and emerging ...    144
- Experience in Credit card/ banking domain with knowledge across customer lifecycle is must;- Candidate ...     46
- Experience in defining and executing professional software engineering best practices for the full ...         33
- Team management / mentor ship experience is must; Should be good at resolving conflicts;- Experience ...       32
- An advanced degree in Math, Computer Science, Statistics, Physics, or a related field (high GPAs ...           29
                                                                                                               ... 
-Relevant professional experience in state-of-the-art Natural

In [ ]:
train.company_name_encoded.nunique()
train.shape

(19802, 9)

In [ ]:
# train.experience.apply(lambda x:x.split("-"))
train.job_type.value_counts()

Analytics    3747
analytics     921
ANALYTICS      77
analytic       32
Analytic       20
Name: job_type, dtype: int64

In [ ]:
df=train.copy()

df.job_description.fillna('None',inplace=True)
df.key_skills.fillna('None',inplace=True)
def exp1(val):
    v=val.split(" ")[0].split("-")
    v1,v2=int(v[0]),int(v[1])
    return (v1+v2)/2
df['exp']=df.experience.apply(exp1)
df['key_skill_len']=df.key_skills.apply(lambda x:len(x.split(",")))


df['exp_low']=df.experience.apply(lambda x:int(x.split("-")[0]))
df['exp_high']=df.experience.apply(lambda x:int(x.split("-")[1].split(" ")[0]))


df['jdescLen']=df.job_description.apply(len)
df['jdsgnLen']=df.job_desig.apply(len)
df['locationLen']=df.location.apply(lambda x:len(x.split(",")))

df.drop(['job_type','Unnamed: 0','experience'],axis=1,inplace=True)
target=df.salary
# target=pd.get_dummies(train.salary)

In [ ]:
df.isnull().sum()
# df.columns

job_description         0
job_desig               0
key_skills              0
location                0
salary                  0
company_name_encoded    0
exp                     0
key_skill_len           0
exp_low                 0
exp_high                0
jdescLen                0
jdsgnLen                0
locationLen             0
dtype: int64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv_jb = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word')
jb =cv_jb.fit_transform(df['job_description'])

cv_jd = TfidfVectorizer(ngram_range=(1,2),stop_words="english", analyzer='word')
jd =cv_jd.fit_transform(df['job_desig'])

cv_key = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word')
ks =cv_key.fit_transform(df['key_skills'])

cv_jbchar = TfidfVectorizer(ngram_range=(1,9),stop_words="english", analyzer='char')
jbchar =cv_jbchar.fit_transform(df['job_description'])

cv_jdchar = TfidfVectorizer(ngram_range=(1,8),stop_words="english", analyzer='char')
jdchar =cv_jdchar.fit_transform(df['job_desig'])

cv_keychar = TfidfVectorizer(ngram_range=(1,9),stop_words="english", analyzer='char')
kschar =cv_keychar.fit_transform(df['key_skills'])

In [ ]:
# loc=np.array(df['location'].values)
# loc.append(np.array(test['location'].values),axis=0)
loc = np.append(df['location'].values, test['location'].values, axis=0)
loc

array(['Delhi NCR(Vikas Puri)', 'Sonepat', 'Delhi NCR', ..., 'Mumbai',
       'Hyderabad', 'Bengaluru'], dtype=object)

In [ ]:
loc.shape

(26403,)

In [ ]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
l.fit(list(set(loc)))
df['location']=l.transform(df['location'])


tot_comp = np.append(df['company_name_encoded'].values, test['company_name_encoded'].values, axis=0)
l_comp=LabelEncoder()
l_comp.fit(list(set(tot_comp)))
df['company_name_encoded']=l_comp.transform(df['company_name_encoded'])

In [ ]:
df.columns

Index(['job_description', 'job_desig', 'key_skills', 'location', 'salary',
       'company_name_encoded', 'exp', 'key_skill_len', 'exp_low', 'exp_high',
       'jdescLen', 'jdsgnLen', 'locationLen'],
      dtype='object')

In [ ]:
from scipy.sparse import csr_matrix
from scipy import sparse
final_features = sparse.hstack((df[['location','company_name_encoded','exp','key_skill_len','jdescLen', 'jdsgnLen', 'locationLen']], jb, jd, ks,jbchar, jdchar, kschar)).tocsr()
# final_features = sparse.hstack((df[['location','company_name_encoded','exp','key_skill_len']], jb, jd, ks)).tocsr()

In [ ]:
final_features

<19802x2102744 sparse matrix of type '<class 'numpy.float64'>'
	with 26948727 stored elements in Compressed Sparse Row format>

In [ ]:
dftest=test.copy()

dftest.job_description.fillna('None',inplace=True)
dftest.key_skills.fillna('None',inplace=True)
def exp1(val):
    v=val.split(" ")[0].split("-")
    v1,v2=int(v[0]),int(v[1])
    return (v1+v2)/2
dftest['exp']=dftest.experience.apply(exp1)
dftest['key_skill_len']=dftest.key_skills.apply(lambda x:len(x.split(",")))

dftest['exp_low']=dftest.experience.apply(lambda x:int(x.split("-")[0]))
dftest['exp_high']=dftest.experience.apply(lambda x:int(x.split("-")[1].split(" ")[0]))

jb_t =cv_jb.transform(dftest['job_description'])
jd_t =cv_jd.transform(dftest['job_desig'])
ks_t =cv_key.transform(dftest['key_skills'])

jb_tchar =cv_jbchar.transform(dftest['job_description'])
jd_tchar =cv_jdchar.transform(dftest['job_desig'])
ks_tchar =cv_keychar.transform(dftest['key_skills'])


dftest['location']=l.transform(dftest['location'])
dftest['company_name_encoded']=l_comp.transform(dftest['company_name_encoded'])

dftest['jdescLen']=dftest.job_description.apply(len)
dftest['jdsgnLen']=dftest.job_desig.apply(len)
dftest['locationLen']=dftest.location.astype(str).apply(lambda x:len(x.split(",")))
dftest.drop(['job_type','experience'],axis=1,inplace=True)

final_features_t = sparse.hstack((dftest[['location','company_name_encoded','exp','key_skill_len','jdescLen', 'jdsgnLen', 'locationLen']], jb_t, jd_t, ks_t,jb_tchar, jd_tchar, ks_tchar)).tocsr()
# final_features_t = sparse.hstack((dftest[['location','company_name_encoded','exp','key_skill_len']], jb_t, jd_t, ks_t)).tocsr()

In [ ]:
final_features_t

<6601x2102744 sparse matrix of type '<class 'numpy.float64'>'
	with 8669216 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train,X_val,y_train,y_val = train_test_split(final_features,target,test_size=0.25,random_state = 1994)

In [ ]:
pip install catboost

     |████████████████████████████████| 67.3MB 47kB/s 


In [ ]:
pip install xgboost

In [ ]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
def train_model(classifier, train_X, train_y, test_X, test_y):
    # fit the training dataset on the classifier
    classifier.fit(train_X, train_y)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(test_X)
    
    return metrics.accuracy_score(predictions, test_y)

In [ ]:
import xgboost
accuracy = train_model(xgboost.XGBClassifier(verbose=True), X_train.tocsc(), y_train, X_val.tocsc(),y_val)
print( "Xgb, Count Vectors: ", accuracy)

Xgb, Count Vectors:  0.4607150070692789


In [ ]:
lr=LogisticRegression(class_weight='balanced',C=2,random_state=1994,n_jobs=-1)
lr.fit(X_train,y_train)
lrpred=lr.predict(X_val)
print(accuracy_score(y_val,lrpred))

0.22217733791153302


In [ ]:
xgb=xgboost.XGBClassifier()

eval_set = [(X_train.tocsc(), y_train),(X_val.tocsc(), y_val)]
xgb.fit(X_train.tocsc(), y_train, eval_metric="mlogloss", eval_set=eval_set, verbose=True,early_stopping_rounds=50)
# make predictions for test data

xgb.fit(X_train.tocsc(),y_train)
xgbpred=xgb.predict(X_val.tocsc())
print(accuracy_score(y_val,xgbpred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

mb=MultinomialNB(alpha=0.5)
mb.fit(final_features,target)
mbpred=mb.predict(final_features_t)

lr=LogisticRegression(class_weight='balanced',C=2,random_state=1994,n_jobs=-1)
lr.fit(final_features,target)
lrpred=lr.predict(final_features_t)

xgb=xgboost.XGBClassifier(verbose=True,n_jobs=-1)
xgb.fit(final_features.tocsc(),target)
xgbpred=xgb.predict(final_features_t.tocsc())

rf=RandomForestClassifier(n_estimators=200)
rf.fit(final_features,target)
rfpred=rf.predict(final_features_t)

cb=CatBoostClassifier()
cb.fit(final_features,target)
cbpred=cb.predict(final_features_t)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
rfpred=rf.predict(X_val)
print(accuracy_score(y_val,rfpred))

In [ ]:
xgbpred

In [ ]:
s=pd.DataFrame({'company_name_encoded':test.company_name_encoded,'salary':xgbpred})
s.head()
s.to_excel('xgbbase9.xlsx',index=False)

In [ ]:
s['salary']=lrpred
s.to_excel('lr.xlsx',index=False)

s['salary']=rfpred
s.to_excel('rf.xlsx',index=False)